In [1]:
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.options.display.max_rows = 999
pd.get_option("display.max_rows")

999

In [3]:
# Control whether to import the original data. 
import_original_data = True

Mila:

OK only four files needed to be corrected. I put them in your Meta4 folder under the name Elviz_Corrected

The remaining four files indeed contained fewer contigs.

Thanks!


In [4]:
import sys
print sys.executable 
print (sys.version)

/usr/bin/python
2.7.5 (default, Mar  9 2014, 22:15:05) 
[GCC 4.2.1 Compatible Apple LLVM 5.0 (clang-500.0.68)]


In [5]:
sns.set(style="whitegrid")

In [6]:
# Import the csv that translates the 127_HOW14 type labels to weeks and replicates.
sample_info = pd.read_csv('./data/sample_meta_info.tsv', sep='\t')


# drop boring columns
del sample_info['week_long']
del sample_info['name']
sample_info.head()

,ID,oxy,rep,week
0,1_LOW4,Low,1,4
1,13_LOW5,Low,1,5
2,25_LOW6,Low,1,6
3,37_LOW7,Low,1,7
4,49_LOW8,Low,1,8


In [7]:
from elviz_utils import IMPORT_DATA_TYPES

In [8]:
# loop over each file, collapse rows that share phylogeny data to the genus label,
# save new .csv files, then append them all together in a separate step. 
#filepath = '/Volumes/Shares/Users/Janet/meta4/160121_elviz_files_updated/'
filepath = './data'
elviz_files = [f for f in os.listdir(filepath) if ".csv" in f]
print len(elviz_files)

88


In [9]:
def reduce_elviz_to_genus_rpk(df):
    '''
    Take in a raw elviz DataFrame and return a Pandas array with a sum at the genus level'''
    # get rid of undesired columns because the data is big.
    columns_to_drop = ['datasetId', 'contigId', 
                       'Average fold', 
                       'Reference GC','Covered percent','Covered bases',
                       #'Plus reads', 'Minus reads', 
                       'Median fold',
                       'Read GC','Complete Lineage', 'MG scaffold_oid']
    for c in columns_to_drop:
        if c in df.columns:
            del df[c]
    # rename NaN at Genus to "other"  
    df.Genus.replace(np.nan, "other", inplace=True)
    
    # calculat RPK: sum the reads that mapped to the plus strand and minus strand
    df['reads per kilobase'] = (df['Plus reads'] + df['Minus reads'])/(df['Length']/1000.)

    ## calculate coverage * length.
    #df['fold_times_length'] = df['Average fold']*df['Length']

    # now we can collapse by shared genus
    #df = df.groupby(['Kingdom','Phylum','Class','Order','Family','Genus'])['fold_times_length'].sum().reset_index()
    df = df.groupby(['Kingdom','Phylum','Class','Order','Family','Genus'])['reads per kilobase'].sum().reset_index()
    # rename our new measure of abundance: 
    df.rename(columns={'reads per kilobase': 'sum of reads per kilobase'}, inplace=True)
    #df.rename(columns={'fold_times_length': 'sum of fold_times_length'}, inplace=True)
    return df 


In [10]:
def id_from_filename(s):
    return re.search('[\w]+_[0-9]+_([0-9]+_[HL]OW[0-9]+).csv', s).group(1)

# test
print elviz_files[0]+": ", id_from_filename(elviz_files[0])
print elviz_files[22]+": ", id_from_filename(elviz_files[22])
print elviz_files[77]+": ", id_from_filename(elviz_files[77])

elviz_1056013_1_LOW4.csv:  1_LOW4
elviz_1056079_33_HOW6.csv:  33_HOW6
elviz_1056244_116_HOW13.csv:  116_HOW13


In [11]:
def project_number_from_filename(s):
    return re.search('[\w]+_([0-9]+)_[0-9]+_[HL]OW[0-9]+.csv', s).group(1)

# test
print elviz_files[0]+": ", project_number_from_filename(elviz_files[0])
print elviz_files[22]+": ", project_number_from_filename(elviz_files[22])
print elviz_files[77]+": ", project_number_from_filename(elviz_files[77])

elviz_1056013_1_LOW4.csv:  1056013
elviz_1056079_33_HOW6.csv:  1056079
elviz_1056244_116_HOW13.csv:  1056244


In [12]:
# calculate abundance.  Note that apply runs the 1st loop item twice:
    # http://stackoverflow.com/questions/21635915/why-does-pandas-apply-calculate-twice
def norm_by_ID(group):
    #print group
    #fold = group['sum of fold_times_length']
    fold = group['sum of reads per kilobase']
    #print rpk.head()
    #print sum(rpk)
    #print ""
    group['sum of reads per kilobase'] = fold/sum(fold)
    return group


In [13]:
def read_and_reduce_elviz_csv(filename, filepath):
    df = pd.read_csv(filepath + '/' + filename, sep=",", dtype=IMPORT_DATA_TYPES, usecols=range(1,20))
    # reduce to one row per genus with reduce_elviz_to_genus_rpk()
    # Need to use DataFrame to convert the seris back to a dataframe if you want to add an ID column.
    df = pd.DataFrame(reduce_elviz_to_genus_rpk(df))
    # add an ID label.  (don't use 'id'; that is a Python built-in.)
    df['ID'] = id_from_filename(filename)
    df['project'] = project_number_from_filename(filename)
    df = pd.merge(df.reset_index(), sample_info)
    df = df.groupby('ID').apply(norm_by_ID)
    # after norm_by_ID is applied, 'Average fold' is now a pooled number.
    # rename column to abundance since we normalized it. 
    df.rename(columns={'sum of reads per kilobase':'abundance'}, inplace=True)
    # sort so most abundant is on top. 
    df.sort_values(by='abundance', axis=0, ascending=False, inplace=True)
    return df

In [14]:
# import the raw or raw-ish data. 
if import_original_data:
    filepath = '/Users/janet/Documents/Lidstrom Lab Work/meta4/160125_Elviz_files_corrected'
    elviz_files = [f for f in os.listdir(filepath) if ".csv" in f]
    print elviz_files[1:4]

['elviz_1056016_2_LOW4.csv', 'elviz_1056019_3_LOW4.csv', 'elviz_1056022_4_LOW4.csv']


In [15]:
if not os.path.exists('./results'):
    os.makedirs('./results/')
if not os.path.exists('./plots'):
    os.makedirs('./plots/')

In [16]:
sample_info.head()

,ID,oxy,rep,week
0,1_LOW4,Low,1,4
1,13_LOW5,Low,1,5
2,25_LOW6,Low,1,6
3,37_LOW7,Low,1,7
4,49_LOW8,Low,1,8


In [17]:
read_and_reduce_elviz_csv(filename = elviz_files[0], filepath=filepath)

,index,Kingdom,Phylum,Class,Order,Family,Genus,abundance,ID,project,oxy,rep,week
642,642,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylobacter,0.275835,1_LOW4,1056013,Low,1,4
494,494,Bacteria,Proteobacteria,Betaproteobacteria,Methylophilales,Methylophilaceae,Methylotenera,0.266572,1_LOW4,1056013,Low,1,4
198,198,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,0.066042,1_LOW4,1056013,Low,1,4
650,650,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,other,0.057879,1_LOW4,1056013,Low,1,4
478,478,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,other,0.036590,1_LOW4,1056013,Low,1,4
462,462,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,Acidovorax,0.024568,1_LOW4,1056013,Low,1,4
509,509,Bacteria,Proteobacteria,Betaproteobacteria,Rhodocyclales,Rhodocyclaceae,Dechloromonas,0.024540,1_LOW4,1056013,Low,1,4
647,647,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylomonas,0.023253,1_LOW4,1056013,Low,1,4
648,648,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylosarcina,0.017382,1_LOW4,1056013,Low,1,4
646,646,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylomicrobium,0.013168,1_LOW4,1056013,Low,1,4


In [18]:
if import_original_data:
    data_reduced = read_and_reduce_elviz_csv(filename = elviz_files[0], filepath=filepath)
    for f in elviz_files[1:]:
        # read and write a csv for that sample
        df_to_add = read_and_reduce_elviz_csv(filename = f, filepath=filepath)
        data_reduced = data_reduced.append(df_to_add)
        print f  # prints filename
        #print data_reduced.head(1)
    print data_reduced.size  
else: 
    data_reduced = pd.read_csv("./results/reduced_data--all_phylogeny_remains.csv")

     index   Kingdom          Phylum                Class             Order  \
642    642  Bacteria  Proteobacteria  Gammaproteobacteria   Methylococcales   
494    494  Bacteria  Proteobacteria   Betaproteobacteria   Methylophilales   
198    198  Bacteria   Bacteroidetes       Flavobacteriia  Flavobacteriales   
650    650  Bacteria  Proteobacteria  Gammaproteobacteria   Methylococcales   
478    478  Bacteria  Proteobacteria   Betaproteobacteria   Burkholderiales   

                Family           Genus  abundance      ID  project  oxy  rep  \
642   Methylococcaceae   Methylobacter   0.275835  1_LOW4  1056013  Low    1   
494   Methylophilaceae   Methylotenera   0.266572  1_LOW4  1056013  Low    1   
198  Flavobacteriaceae  Flavobacterium   0.066042  1_LOW4  1056013  Low    1   
650   Methylococcaceae           other   0.057879  1_LOW4  1056013  Low    1   
478     Comamonadaceae           other   0.036590  1_LOW4  1056013  Low    1   

     week  
642     4  
494     4  
198     

In [19]:
data_reduced.head()

,index,Kingdom,Phylum,Class,Order,Family,Genus,abundance,ID,project,oxy,rep,week
642,642,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylobacter,0.275835,1_LOW4,1056013,Low,1,4
494,494,Bacteria,Proteobacteria,Betaproteobacteria,Methylophilales,Methylophilaceae,Methylotenera,0.266572,1_LOW4,1056013,Low,1,4
198,198,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,0.066042,1_LOW4,1056013,Low,1,4
650,650,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,other,0.057879,1_LOW4,1056013,Low,1,4
478,478,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,other,0.036590,1_LOW4,1056013,Low,1,4


In [32]:
# extract out the project number and ID:
id_and_proj = data_reduced[['ID', 'project']].drop_duplicates()
sample_info_augmented = sample_info.merge(right=id_and_proj, how='left')


In [35]:
print sample_info_augmented.shape
sample_info_augmented.head()

(88, 5)


,ID,oxy,rep,week,project
0,1_LOW4,Low,1,4,1056013
1,13_LOW5,Low,1,5,1056037
2,25_LOW6,Low,1,6,1056061
3,37_LOW7,Low,1,7,1056085
4,49_LOW8,Low,1,8,1056109


In [37]:
sample_info_augmented.sort_values(by='project',)
sample_info_augmented.to_csv('data/sample_info.tsv', sep='\t', index=False, )

,ID,oxy,rep,week,project
0,1_LOW4,Low,1,4,1056013
11,2_LOW4,Low,2,4,1056016
22,3_LOW4,Low,3,4,1056019
33,4_LOW4,Low,4,4,1056022
44,7_HOW4,High,1,4,1056025
55,8_HOW4,High,2,4,1056028
66,9_HOW4,High,3,4,1056031
77,10_HOW4,High,4,4,1056034
1,13_LOW5,Low,1,5,1056037
12,14_LOW5,Low,2,5,1056040


In [ ]:
# Should have only 1 row per Id now. 
data_reduced[data_reduced['Genus']=='Methylophilus']

if import_original_data:
    data_reduced.to_csv("./results/reduced_data--all_phylogeny_remains.csv", index=False)

In [ ]:
data_reduced.columns

In [ ]:
# make sure the abundances all sum to 1. 
for t, d in data_reduced.groupby('ID'):
    print t, ":", d['abundance'].sum()

In [ ]:
data_reduced.tail()

In [ ]:
# check number of sample imported.  Dave had a problem with the raw files 1/16/2015 using Python 3. 
len(data_reduced.ID.unique())

In [ ]:
data_reduced['sample_name'] = 'replicate '+ data_reduced['rep'].astype(str) +": " + data_reduced['oxy'] + ' O2'
print data_reduced['sample_name'].unique()

In [ ]:
data_reduced.tail()

In [ ]:
# Write an excel for Mila.  One notebook for replicate.  Each page is a different ID/week. 
by_repl_and_week = data_reduced.groupby(['rep','week','oxy'])#.to_excel(writer,sheet_name='Sheet1')

In [ ]:
# The groupby returns tuples of the conditions and the data in a tuple. 
#for (rep, week, oxy), d in by_repl_and_week:
#    print "rep_" + str(rep)
#    print "week_" + str(week)
#    print oxy
#    print ""

In [ ]:
data_reduced['rep'].unique()

In [ ]:
# write same dictionary in a loop
excel_files = {}
for ox in data_reduced['oxy'].unique():
    for re in data_reduced['rep'].unique():
        print ox, re
        excel_files[(ox, re)] =  'elviz_binned--{}O2_rep{}.xlsx'.format(ox, re)
  
print excel_files

In [ ]:
# Turn off re-writ here:

write_excel = True
if import_original_data and write_excel:
#if write_excel:
    # make an excel writer for each file. 
    writer_dict = {}
    for oxy in data_reduced['oxy'].unique():
        for rep in data_reduced['rep'].unique():
            #print ox, re
            filename = 'elviz--{}O2_rep_{}.xlsx'.format(oxy, rep)
            #print filename
            writer_dict[(oxy, rep)] = pd.ExcelWriter(filename, engine='xlsxwriter')
            #print writer_dict
    print writer_dict
    
    # The groupby returns tuples of the conditions. 
    by_repl_and_week = data_reduced.groupby(['rep','week','oxy'])#.to_excel(writer,sheet_name='Sheet1')  
    for (rep, week, oxy), d in by_repl_and_week:
        # use the writer that matches the replicate:
        print rep, week, oxy
        writer = writer_dict[(oxy, rep)]
        sheet_name = oxy + '_O2' +"_rep_" + str(rep)+"_week_" + str(week)
        print sheet_name
        d.reset_index()
        #print d.columns
        del d['index']
        #print d.columns
        d.to_excel(writer, sheet_name=sheet_name, index=False)
        #writer.save()
        print ""
    
    # close each writer.  This saves them. 
    for w_dict in writer_dict.values():
        w_dict.close()


In [ ]:
data_reduced.week.head()

In [ ]:
data_reduced.head()

In [ ]:
# Another check that we have each ID's abundances summing to 1. 
for t, d in data_reduced.groupby('ID'):
    if d['abundance'].sum() > 1.0001:
        print t, ":", d['abundance'].sum()

In [ ]:
data_reduced.head()

In [ ]:
# Further reduce data so only Genus is preserved. 
# drop the reads per kilobase or we will get more than 1 row for other.
data_reduced_genus = data_reduced.copy()
#del data_reduced_genus['reads per kilobase']
print data_reduced_genus.head()
data_reduced_genus = data_reduced_genus.groupby(['ID','rep','week','oxy','Genus']).sum().reset_index()
data_reduced_genus.sort_values(by=['rep', 'abundance'], inplace=True, ascending=False)
data_reduced_genus.tail(8)

In [ ]:
data_reduced_genus.columns

In [ ]:
data_reduced_genus.head()

In [ ]:
print data_reduced_genus[['rep','week', 'ID']].drop_duplicates()

In [ ]:
data_reduced_genus.head(3)

In [ ]:
by_repl_and_week_Genus = data_reduced_genus.groupby(['rep','week','oxy'])#.to_excel(writer,sheet_name='Sheet1')
# check that replicates and week are in the right order: 
#for tup, d in by_repl_and_week_Genus:
#    print tup

In [ ]:
# Turn off re-writ here:
write_excel = True

if write_excel:
    # make an excel writer for each file. 
    writer_dict = {}
    for oxy in data_reduced['oxy'].unique():
        for rep in data_reduced['rep'].unique():
            #print ox, re
            filename = 'elviz--Genus_only--{}O2_rep_{}.xlsx'.format(oxy, rep)
            #print filename
            writer_dict[(oxy, rep)] = pd.ExcelWriter(filename, engine='xlsxwriter')
            #print writer_dict
    print writer_dict
    
    # The groupby returns tuples of the conditions. 
    by_repl_and_week = data_reduced_genus.groupby(['rep','week','oxy'])#.to_excel(writer,sheet_name='Sheet1')  
    for (rep, week, oxy), d in by_repl_and_week:
        # use the writer that matches the replicate:
        print rep, week, oxy
        writer = writer_dict[(oxy, rep)]
        sheet_name = oxy + '_O2' +"_rep_" + str(rep)+"_week_" + str(week)
        print sheet_name
        d.reset_index(inplace=True)
        del d['index']
        #print d.columns
        del d['level_0']
        d.to_excel(writer, sheet_name=sheet_name, index=False)
        #writer.save()
        print ""
    
    # close each writer.  This saves them. 
    for w_dict in writer_dict.values():
        w_dict.close()


In [ ]:
# Demo of simple plot.  Trim rows with abundance some percent.  Plot only one week/oxygen combo. 
def plot_by_abundance(data, abundance):
    d_plot = data[data['abundance'] > abundance]
    d_plot = pd.pivot_table(d_plot, values='abundance', index=['week'], columns='Genus') #.reset_index()
    print d_plot.head()
    d_plot.plot(kind='bar',stacked=True)
    sns.despine()
    #plt.legend(loc=2)
plot_by_abundance(data=data_reduced_genus[
        np.logical_and(
            data_reduced_genus['rep']==1, 
            data_reduced_genus['oxy']=="Low")], abundance=0.02)

In [ ]:
# Begin to generalize.  Want to be able to subset by oxygen and replicate. 
def subset_data_by_rep_and_oxy(data, rep, oxy):
    return data[
        np.logical_and(
            data_reduced_genus['rep']==rep, 
            data_reduced_genus['oxy']==oxy)]

In [ ]:
# Make an individual plot with the new subset_data_by_rep_and_oxy() function
def plot_by_abundance(data, abundance):
    d_plot = data[data['abundance'] > abundance]
    d_plot = pd.pivot_table(d_plot, values='abundance', index=['week'], columns='Genus') #.reset_index()
    #print d_plot.head()
    d_plot.plot(kind='bar',stacked=True)
    sns.despine()
    #plt.legend(loc=2)
plot_by_abundance(subset_data_by_rep_and_oxy(data= data_reduced_genus, rep=1, oxy='High'), abundance=0.02)

In [ ]:
# set up a figure with four subplots. 
fig, axs = plt.subplots(2, 4, figsize=(8, 6))  # ax belongs to fig. 
print fig
print axs

In [ ]:
g = data_reduced_genus.groupby(['rep', 'oxy'])
for t, dg in g:
    print t #, dg
    #print t[0]

In [ ]:
row_dict = {(1, 'High') : 2,
                (1, 'Low')  : 1,
                (2, 'High') : 2,
                (2, 'Low')  : 1,
                (3, 'High') : 2,
                (3, 'Low')  : 1,
                (4, 'High') : 2,
                (4, 'Low')  : 1}

In [ ]:
print axs
print ""
axs[(1-1)][2-1]

In [ ]:
#axd = {1:axs[0], 2:axs[1], 3:axs[2], 4:axs[3]}
title_list = ''

for t, df in g:
    #print t
    #print 'plot row:', row_dict[t]
    #print 'plot column:', t[0]
    df.plot(kind='scatter', ax=axs[row_dict[t]-1][t[0]-1], x='week', y='abundance', title='meow')

sns.despine()  # Removes the boxes around the plots.

In [ ]:
fig

In [ ]:
# set up a figure with four subplots. 
fig, axs = plt.subplots(2, 4, figsize=(12, 10))  # ax belongs to fig. 
#print fig
#print axs

# set up groupby 

d_plot = data_reduced_genus[data_reduced_genus['abundance'] > 0.05]
g = d_plot.groupby(['rep', 'oxy'])

#axd = {1:axs[0], 2:axs[1], 3:axs[2], 4:axs[3]}
title_list = ''

row_dict = {(1, 'High') : 2, (2, 'High') : 2, (3, 'High') : 2, (4, 'High') : 2,
            (1, 'Low')  : 1, (2, 'Low')  : 1, (3, 'Low')  : 1, (4, 'Low')  : 1}

for tup, df in g:
    # cast the data
    #print tup # tuple
    data_pivoted = pd.pivot_table(df, values='abundance', 
                                        index=['week'],  # drops all the other columns. 
                                        columns='Genus') 
    #print data_pivoted.head()
    #df.groupby('Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'ID', 'oxy', 'rep')
    #print axs[row_dict[tup]-1][tup[0]-1]
    data_pivoted.plot(kind='bar', ax=axs[row_dict[tup]-1][tup[0]-1], stacked=True)  # don't specify x b/c there is only 1 index. 


In [ ]:
# fig  # shows the figure.

In [ ]:
## We can get the last one out, but we want all of them out and want them to have shared colors. 
# set up a figure with four subplots. 
fig, axs = plt.subplots(2, 4, figsize=(12, 10))  # ax belongs to fig. 
#print fig
print axs

# set up groupby 

d_plot = data_reduced_genus[data_reduced_genus['abundance'] > 0.05]
g = d_plot.groupby(['rep', 'oxy'])

#axd = {1:axs[0], 2:axs[1], 3:axs[2], 4:axs[3]}
title_list = ''

row_dict = {(1, 'High') : 2, (2, 'High') : 2, (3, 'High') : 2, (4, 'High') : 2,
            (1, 'Low')  : 1, (2, 'Low')  : 1, (3, 'Low')  : 1, (4, 'Low')  : 1}

for tup, df in g:
    # cast the data
    #print tup # tuple
    data_pivoted = pd.pivot_table(df, values='abundance', 
                                        index=['week'],  # drops all the other columns. 
                                        columns='Genus') 
    #print data_pivoted.head()
    #df.groupby('Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'ID', 'oxy', 'rep')
    #print axs[row_dict[tup]-1][tup[0]-1]
    data_pivoted.plot(kind='bar', ax=axs[row_dict[tup]-1][tup[0]-1], stacked=True)  # don't specify x b/c there is only 1 index. 

plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.1,1,1),
            bbox_transform = plt.gcf().transFigure )            
#legend = plt.legend(loc='upper center', shadow=True, fontsize='x-large')   
#fig.legend(bbox_to_anchor=(1.05, 0), loc='lower left', borderaxespad=0.)
#         # it will place the legend on the outer right-hand side of the last axes    

In [ ]:
import itertools
methods=['method 1', 'method2', 'method 3', 'method 4']
times = range(0, 100, 10)
data = pd.DataFrame(list(itertools.product(methods, times, times)))
data.columns = ['method', 'dtsi','rtsi']
data['nw_score'] = np.random.sample(data.shape[0])
print data.head()

In [ ]:
# preview of pivot that happens:  (note use of pivot_table, not pivot.  Aggregating across duplicate rows somehow.)
data.pivot_table(index="dtsi", columns='rtsi', values='nw_score').head()

In [ ]:
data['dtsi'].unique()  # x-values
# rtsi are y-values
# cast 

In [ ]:
data_reduced_genus.head()

In [ ]:
# Make a heat map.  Then we aren't confused by colors, and overwhelmed by the # of labels. 
d_plot = data_reduced_genus[data_reduced_genus['abundance'] > 0.05]

def facet_heatmap(data, color, **kws):
    data = data.pivot(index='Genus', columns='week', values='abundance')
    sns.heatmap(data, cmap='Blues', **kws)  # <-- Pass kwargs to heatmap

with sns.plotting_context(font_scale=5.5):
    g = sns.FacetGrid(d_plot[d_plot['oxy']=='Low'], 
                      col="rep", size=3, aspect=1)

cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes

g = g.map_dataframe(facet_heatmap,
                    cbar_ax=cbar_ax,
                    vmin=0, vmax=1)  # <-- Specify the colorbar axes and limits

g.set_titles(col_template="{col_name}", fontweight='bold', fontsize=18)
g.fig.subplots_adjust(right=.9)  # <-- Add space so the colorbar doesn't overlap the plot

In [ ]:
def filter_by_abundance(data, column, high, low):
    '''Return only rows where genera have one value of abundance in range(low, high)'''
    species_to_keep = data[(data[column] <= high) & (data[column] >= low)]['Genus'].unique()
    print species_to_keep[0:5]# 
    return data[data['Genus'].isin(species_to_keep)]
    #return data[data['Genus'] in species_to_keep]

filter_by_abundance(data=data_reduced_genus, column='abundance', high=1, low=0.5).head()

In [ ]:
# Make a heat map.  Then we aren't confused by colors, and overwhelmed by the # of labels. 
d_plot = filter_by_abundance(data=data_reduced_genus, column='abundance', high=.8, low=0.1)

def facet_heatmap(data, color, **kws):
    data = data.pivot(index='Genus', columns='week', values='abundance')
    sns.heatmap(data, cmap="YlGnBu", **kws)  # <-- Pass kwargs to heatmap  cmap used to be 'Blue'

with sns.plotting_context(font_scale=7):
    g = sns.FacetGrid(d_plot[d_plot['oxy']=='Low'], 
                      col="rep", size=3, aspect=1)

cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes

g = g.map_dataframe(facet_heatmap,
                    cbar_ax=cbar_ax,
                    vmin=0, vmax=max(d_plot.abundance))  # <-- Specify the colorbar axes and limits

g.set_titles(col_template="{col_name}", fontweight='bold', fontsize=18)
g.fig.subplots_adjust(right=.9)  # <-- Add space so the colorbar doesn't overlap the plot

In [ ]:
def plot_heatmap(data, high, low, oxy, rep): 
    # get rid of oxygen levels and replicates if specified.
    if oxy is not 'all':
        print "keep only {} oxygen samples".format(oxy)
        data = data[data['oxy']==oxy]
    if rep is not 'all':
        print "keep only replicate levels:", rep
        data = data[data['rep'].isin(rep)]        
    data = filter_by_abundance(data=data, column='abundance', high=high, low=low)
    data['facet_replicate'] = 'replicate ' + data['rep'].astype(str)
    
    # make height of the plot a function of the number of rows (Genera):
    num_data_rows = len(data['Genus'].unique())
    plot_size = 2 +  num_data_rows/7
    plot_aspect = 2
    if num_data_rows > 6:
        plot_aspect = .85
    if num_data_rows > 9:
        plot_aspect = .65
    if num_data_rows > 9:
        plot_aspect = .6
    
    def facet_heatmap(data, color, **kws):
        data = data.pivot(index='Genus', columns='week', values='abundance')
        sns.heatmap(data, cmap="YlGnBu", **kws)  # <-- Pass kwargs to heatmap  cmap used to be 'Blue'
    
    with sns.plotting_context(font_scale=7):
        g = sns.FacetGrid(data, col='facet_replicate', size=plot_size, aspect=plot_aspect)
    
    cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes
    
    g = g.map_dataframe(facet_heatmap,
                        cbar_ax=cbar_ax,
                        vmin=0, vmax=max(data.abundance))  # <-- Specify the colorbar axes and limits
    
    g.set_titles(col_template="{col_name}", fontweight='bold', fontsize=18)
    g.set_axis_labels('week')
    g.fig.subplots_adjust(right=.9)  # <-- Add space so the colorbar doesn't overlap the plot
    
    # add a supertitle. 
    plt.subplots_adjust(top=0.80)
    supertitle = str(low) + ' < abundance < ' + str(high) + ', {} oxygen'.format(oxy) 
    g.fig.suptitle(supertitle, size=14)
    
    # write a filename and save.
    plot_dir = './plots/'
    filename = oxy + "_oxygen--{0}_to_{1}_abundance".format(low,high)
    print 'filename:', filename
    g.savefig(plot_dir + filename + '.pdf')

# demo:
plot_heatmap(data=data_reduced_genus, high=0.6, low = 0.5, oxy='High', rep='all')

In [ ]:
plot_heatmap(data=data_reduced_genus, high=1, low = 0.1, oxy='High', rep='all')

In [ ]:
plot_heatmap(data=data_reduced_genus, high=.5, low = 0.02, oxy='Low', rep='all')

In [ ]:
# Loop through the ranges we want. 
oxy_conditions = ['Low', 'High']
range_pairs = [(0.01, 1), (0.05, 1), (0.1, 1), (0.5, 1)]
for o in oxy_conditions:
    for r in range_pairs:
        #print "oxygen:", o
        #print "range:", r
        #print "range (low):", r[0]
        plot_heatmap(data=data_reduced_genus, low =r[0], high=r[1], oxy=o, rep='all')

In [ ]:
# Loop through the ranges we want. 
oxy_conditions = ['Low', 'High']
max_abundance = 0.6
range_pairs = [(0.01, max_abundance), (0.05, max_abundance), (0.1, max_abundance), (0.5, max_abundance)]
for o in oxy_conditions:
    for r in range_pairs:
        #print "oxygen:", o
        #print "range:", r
        #print "range (low):", r[0]
        plot_heatmap(data=data_reduced_genus, low =r[0], high=r[1], oxy=o, rep='all')